## Imports

In [1]:
import copy
import os
import random
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical

# Lab 6: Project CERN

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Download, cache and extract bird song data

In [3]:
dataset_dir = Path('datasets')
remote_dataset_file='/content/drive/My Drive/dataset_v2.tar.gz'
#data_for_processing = keras.utils.get_file(myFile, 'file://'+fullPath)
if not (dataset_dir/'testing_list.txt').exists(): # Assume dataset already downloaded/extracted if testing list is present
    get_file(None, "file://" + remote_dataset_file,
                    extract=True,
                    cache_dir='.',
                    cache_subdir=dataset_dir)

## Load Bird song data

In [4]:
# Classes to handle, ordered by label
CLASSES = ['alauda_arvensis', 'cuculus_canorus', 'delichon_urbicum', 'emberiza_cirlus', 'emberiza_citrinella', 'falco_tinnunculus', 'muscicapa_striata', 'sylvia_borin', 'tachybaptus_ruficollis', 'tyto_alba']

with (dataset_dir/'testing_list.txt').open() as f:
    testing_list = f.read().splitlines()

x_train = []
y_train = []
x_test = []
y_test = []

for recording in dataset_dir.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        print("avoid")
        continue
    label = CLASSES.index(recording.parent.name) # Assign class number
    
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer
        
    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((16000, 1)) # Resize to 1s (16kHz) with zero-padding, 1 channel

    if str(recording.relative_to(dataset_dir)) in testing_list: # Assign to test set if file in test list
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

## Normalize data

In [5]:
# Use train set as a reference for mean/std
x_mean = x_train.mean()
x_std = x_train.std()

x_train -= x_mean
x_test  -= x_mean
x_train /= x_std
x_test  /= x_std

## Export small dataset (250 random vectors)

In [6]:
perms = np.random.permutation(len(y_test))[0:250]
x_test_250 = x_test[perms]
y_test_250 = y_test[perms]
np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model M5

In [7]:
model = Sequential()
model.add(Input(shape=(16000, 1)))

### SMALLEST

model.add(MaxPool1D(pool_size=10))
model.add(Conv1D(filters=32, kernel_size=80, strides=4, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=1))

model.add(Flatten())
model.add(Dense(units=10))
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=10e-4)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d (MaxPooling1D  (None, 1600, 1)          0         
 )                                                               
                                                                 
 conv1d (Conv1D)             (None, 381, 32)           2592      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 95, 32)           0         
 1D)                                                             
                                                                 
 conv1d_1 (Conv1D)           (None, 93, 32)            3104      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 23, 32)           0         
 1D)                                                             
                                                        

## Train model

In [8]:
model.fit(x_train, y_train, epochs=10, batch_size=150, validation_data=(x_test, y_test))

Epoch 1/10
155/155 [==============================] - 19s 41ms/step - loss: 2.1514 - categorical_accuracy: 0.2368 - val_loss: 1.9842 - val_categorical_accuracy: 0.3186
Epoch 2/10
155/155 [==============================] - 2s 14ms/step - loss: 1.8652 - categorical_accuracy: 0.3770 - val_loss: 1.7520 - val_categorical_accuracy: 0.4212
Epoch 3/10
155/155 [==============================] - 2s 15ms/step - loss: 1.6822 - categorical_accuracy: 0.4433 - val_loss: 1.5861 - val_categorical_accuracy: 0.4935
Epoch 4/10
155/155 [==============================] - 2s 15ms/step - loss: 1.5622 - categorical_accuracy: 0.4824 - val_loss: 1.5521 - val_categorical_accuracy: 0.4806
Epoch 5/10
155/155 [==============================] - 2s 14ms/step - loss: 1.4724 - categorical_accuracy: 0.5118 - val_loss: 1.4743 - val_categorical_accuracy: 0.5196
Epoch 6/10
155/155 [==============================] - 3s 17ms/step - loss: 1.4003 - categorical_accuracy: 0.5376 - val_loss: 1.3877 - val_categorical_accuracy: 0.55

## Evaluate model on test dataset

In [9]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

312/312 - 1s - loss: 1.2750 - categorical_accuracy: 0.5951 - 1s/epoch - 4ms/step
312/312 [==============================] - 1s 3ms/step
tf.Tensor(
[[560  16 120  27  60   6  10  90  43  28]
 [  7 730 124   7  21  11  32  16  17  29]
 [ 21  38 772  17  13  17  64  33   8  40]
 [ 27  10 205 507  58  20  37  44  26  44]
 [ 18   9 121  47 442  44  73 132  25  68]
 [  9  23 117  18 131 397  83 116  44  80]
 [  2  32 209  23  79  38 430  43  29  83]
 [ 46   2  62  13  86  18  13 819   1  10]
 [  9  14  87  90  50  46 105  47 458  51]
 [  8  15 107   8  20   6  34   0   6 816]], shape=(10, 10), dtype=int32)


## Evaluate model on small dataset

In [10]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 1.1779 - categorical_accuracy: 0.6160 - 145ms/epoch - 18ms/step
8/8 [==============================] - 0s 3ms/step
tf.Tensor(
[[ 8  1  0  1  1  0  1  2  2  1]
 [ 0 24  2  0  0  0  1  0  0  1]
 [ 0  0 23  0  0  0  2  2  0  1]
 [ 0  0  5 16  1  0  1  1  0  0]
 [ 1  0  4  1  7  0  1  4  1  1]
 [ 0  1  4  0  4  7  4  4  3  0]
 [ 0  1 10  0  1  0  7  1  0  1]
 [ 1  0  3  0  1  0  0 25  0  0]
 [ 0  0  1  4  1  1  3  2 10  3]
 [ 0  0  2  0  1  0  0  0  0 27]], shape=(10, 10), dtype=int32)


## Save trained model

In [11]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [12]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [13]:
!pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Generate C code for the trained model with 16-bit fixed-point representation

In [14]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

———————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                           | Layer                            | Outputs                         
———————————————————————————————————————————————————————————————————————————————————————————————————————
                                 | input_1                          | max_pooling1d                   
-------------------------------------------------------------------------------------------------------
input_1                          | max_pooling1d                    | conv1d                          
-------------------------------------------------------------------------------------------------------
max_pooling1d                    | conv1d                           | max_pooling1d_1                 
-------------------------------------------------------------------------------------------------------
conv1d                           | max_pooling1d_1                  

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [16]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv

g++: error: main.cpp: No such file or directory
/bin/bash: ./gsc_fixed: No such file or directory
